In [272]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By
import re


# -- API --
import re
import json
import requests
from tqdm import tqdm

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [287]:
# Open chrome with chromeDriver
# Check version your Chrome and download the same version chromeDriver
# https://chromedriver.chromium.org/downloads

driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")

C:\Users\Admin\AppData\Local\Temp\ipykernel_9660\1752774495.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")


In [292]:
# Auto Login or Pass the block from Shopee

def login_account(driver):
    if(len(driver.find_elements(by=By.NAME, value='loginKey'))):
        _username = 'will@socialyze.asia'
        _password = 'Quan0388925887'

        username_field = driver.find_element(by=By.NAME, value='loginKey')
        username_field.send_keys(_username)

        password_field = driver.find_element(by=By.NAME, value='password')
        password_field.send_keys(_password)

        sleep(0.5)

        login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
        login_button.click()

        sleep(10)

In [275]:
link_products = []

number_of_page_items = 10 # Depend on the number of product page

# Get link products from link shop items
for i in range(number_of_page_items): 
    link_list_products = f"https://shopee.vn/kemnghiavy90?page={i}&sortBy=pop"
    driver.get(link_list_products)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    driver.execute_script("window.scrollTo(0, 1000);")
    sleep(1)
    xpath_hrefs = driver.find_elements(by=By.XPATH, value='//div[@class="shop-search-result-view__item col-xs-2-4"]/a')

    for xpath_href in xpath_hrefs:
        link_products.append(xpath_href.get_attribute('href'))

    if(len(xpath_hrefs) < 10):
        break

In [276]:
len(set(link_products))

20

In [277]:
pd.DataFrame(link_products, columns = ['link']).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220907_KemNghia/link_products_kemnghiavy90.csv', index = False)

In [278]:
link_products = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220907_KemNghia/link_products_kemnghiavy90.csv')
link_products = link_products.link
link_products

0     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
1     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-D04-m...
2     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
3     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
4     https://shopee.vn/K%C3%A9o-ngh%C4%A9a-t%E1%BB%...
5     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
6     https://shopee.vn/B%E1%BB%99-r%C3%A1y-tai-k%E1...
7     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
8     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
9     https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
10    https://shopee.vn/S%E1%BB%A7i-da-k%E1%BB%81m-n...
11    https://shopee.vn/k%E1%BB%81m-ngh%C4%A9a-c%E1%...
12    https://shopee.vn/k%E1%BB%81m-ngh%C4%A9a-c%E1%...
13    https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-c%E1%...
14    https://shopee.vn/K%E1%BB%81m-Ngh%C4%A9a-M04-c...
15    https://shopee.vn/K%C3%A9o-Ngh%C4%A9a-c%E1%BA%...
16    https://shopee.vn/B%E1%BB%99-k%E1%BB%81m-ngh%C...
17    https://shopee.vn/K%E1%BB%81m-ngh%C4%A9a-b

Get cmt from link_products

In [279]:
# Get comment counts

def get_cmt_counts(driver):
    check_cmt_counts = driver.find_element(by=By.XPATH, value='//div[@class="product-rating-overview__filter product-rating-overview__filter--with-comment"]').text
    cmt_counts = 0
    if('k' in check_cmt_counts):
        get_number_in_check_cmt_counts = re.findall('\d', check_cmt_counts)
        if(len(get_number_in_check_cmt_counts) == 1): # only [1,2,...,9]k
            cmt_counts = int(get_number_in_check_cmt_counts[0])*1000
        elif(len(get_number_in_check_cmt_counts) == 2): # >= 1k and <= 9k9
            cmt_counts = int(get_number_in_check_cmt_counts[0])*1000 + int(get_number_in_check_cmt_counts[1])*100
        elif(len(get_number_in_check_cmt_counts) == 3): # >= 10k and <= 99k9
            cmt_counts = int(get_number_in_check_cmt_counts[0])*10000 + int(get_number_in_check_cmt_counts[1])*1000 + int(get_number_in_check_cmt_counts[2])*100
    elif('k' not in check_cmt_counts):
        if(len(re.findall('\d', check_cmt_counts)) == 2): # 10 -99
            cmt_counts = int(re.findall('\d', check_cmt_counts)[0])*10 + int(re.findall('\d', check_cmt_counts)[1])
        elif(len(re.findall('\d', check_cmt_counts)) == 3): # 100 - 999
            cmt_counts = int(re.findall('\d', check_cmt_counts)[0])*100 + int(re.findall('\d', check_cmt_counts)[1])*10 + int(re.findall('\d', check_cmt_counts)[2])
        elif(len(re.findall('\d', check_cmt_counts)) == 1): # 0 - 9
            cmt_counts = int(re.findall('\d', check_cmt_counts)[0])
    return cmt_counts

In [280]:
# Crawl data from multiple threads

def startCrawl(product_links, list_product):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")
    check_point_link_product = 1
    for link_product in product_links:

        driver.get(link_product)
        sleep(4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Check block login
        login_account(driver)

        sleep(1)
        driver.execute_script("window.scrollTo(0, 500);")

        print(driver.find_element(by=By.XPATH, value='//div[@class="_2rQP1z"]').text)

        if('kềm nghĩa' in (driver.find_element(by=By.XPATH, value='//div[@class="_2rQP1z"]').text).lower()):
            login_account(driver)

            driver.execute_script("window.scrollTo(0, 1000);")
            sleep(1)
            driver.execute_script("window.scrollTo(0, 500);")
            sleep(0.5)
            driver.execute_script("window.scrollTo(0, 1500);")
            sleep(0.5)

            cmt_counts = get_cmt_counts(driver)

            if(cmt_counts < 8): # Check link if the page has only 1 comment page (1 comment page shopee has only 6 comments)
                xpath_names = driver.find_elements(by=By.XPATH, value='//a[@class="shopee-product-rating__author-name"]')
                xpath_cmts = driver.find_elements(by=By.XPATH, value='//div[@class="_280jKz"]')
                xpath_review_ratings = driver.find_elements(by=By.CSS_SELECTOR, value = '.shopee-product-rating')
                xpath_date_cmts = driver.find_elements(by=By.XPATH, value='//div[@class="shopee-product-rating__time"]')

                for i in range(len(xpath_names)):
                    login_account(driver)

                    Product_Infos = {}

                    shopid_productid = link_products[0].split('?sp_atk')[0].split('-i.')[1].split('.')

                    Product_Infos['shop_ids'] = shopid_productid[0]
                    Product_Infos['product_ids'] = shopid_productid[1]

                    try:
                        Product_Infos['names'] = xpath_names[i].text
                    except:
                        continue

                    try:
                        Product_Infos['cmts'] = xpath_cmts[i].text
                    except:
                        continue

                    try:
                        Product_Infos['review_ratings'] = len(xpath_review_ratings[i].find_elements(by=By.CSS_SELECTOR, value = '.icon-rating-solid'))
                    except:
                        continue


                    Product_Infos['link_crawlings'] = link_product

                    try:
                        Product_Infos['date_cmts'] = xpath_date_cmts[i].text
                    except:
                        continue

                    Product_Infos['date_crawl'] = str(date.today())
                    list_product.append(Product_Infos)
                    sleep(1)
            else: # Check link if the page has 2 comment pages or more
                check_point = 1 # Check number of crawled link


                login_account(driver)

                for check_point in range(get_cmt_counts(driver) // 6):

                    driver.execute_script("window.scrollTo(0, 1000);")
                    sleep(0.5)

                    login_account(driver)

                    driver.execute_script("window.scrollTo(0, 1500);")
                    sleep(0.5)

                    xpath_names = driver.find_elements(by=By.XPATH, value='//a[@class="shopee-product-rating__author-name"]')
                    xpath_cmts = driver.find_elements(by=By.XPATH, value='//div[@class="_280jKz"]')
                    xpath_review_ratings = driver.find_elements(by=By.CSS_SELECTOR, value = '.shopee-product-rating')
                    xpath_date_cmts = driver.find_elements(by=By.XPATH, value='//div[@class="shopee-product-rating__time"]')
                    sleep(1)

                    for i in range(len(xpath_names)):
                        login_account(driver)

                        Product_Infos = {}

                        shopid_productid = link_products[0].split('?sp_atk')[0].split('-i.')[1].split('.')

                        Product_Infos['shop_ids'] = shopid_productid[0]
                        Product_Infos['product_ids'] = shopid_productid[1]

                        try:
                            Product_Infos['names'] = xpath_names[i].text
                        except:
                            continue

                        try:
                            Product_Infos['cmts'] = xpath_cmts[i].text
                        except:
                            continue

                        try:
                            Product_Infos['review_ratings'] = len(xpath_review_ratings[i].find_elements(by=By.CSS_SELECTOR, value = '.icon-rating-solid'))
                        except:
                            continue


                        Product_Infos['link_crawlings'] = link_product

                        try:
                            Product_Infos['date_cmts'] = xpath_date_cmts[i].text
                        except:
                            continue

                        Product_Infos['date_crawl'] = str(date.today())
                        list_product.append(Product_Infos)

                        button_continue = driver.find_element(by=By.XPATH, value='//button[@class="shopee-icon-button shopee-icon-button--right "]')
                        button_continue.click()
                    print(str(check_point) + ' --- ' + str(len(list_product)))

                    check_point += 1
           
        else:
            print('Khong co keyword "Kem Nghia" trong ten san pham')
            pass
    
        print("Link product thu ", check_point_link_product)
        check_point_link_product += 1

In [281]:
import threading
import numpy as np

list_threads = []
list_product = []
num_thread = 3 # Number of threads
list_partitions = [partition for partition in np.array_split(link_products, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, list_product)))

In [282]:
for thread in list_threads:
    thread.start()  
    time.sleep(5)       

C:\Users\Admin\AppData\Local\Temp\ipykernel_9660\2815204012.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")


Kềm nghĩa cắt da D18


In [ ]:
for thread in list_threads:
    thread.join()  

In [284]:
len(list_product)

4

In [285]:
pd.DataFrame(list_product).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220907_KemNghia/cmts_kemnghiavy90.csv', index=False)